# PDFPLUMBER

In [ ]:
import pdfplumber
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
import random
import os
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk


In [42]:
def hasTargetHeaders(tabla, encabezados_objetivo):
    """
    Verifica si la primera fila de una tabla PDF contiene todos los encabezados esperados.

    Parámetros:
        tabla (list[list[str]]): Lista de filas (cada fila es una lista de celdas).
        encabezados_objetivo (set[str]): Conjunto de encabezados requeridos (en minúsculas).

    Retorna:
        bool: True si la tabla contiene todos los encabezados buscados, False en caso contrario.
    """
    if not tabla or len(tabla) == 0:
        return False
    encabezados = [str(celda).strip().lower() for celda in tabla[0] if celda]
    return encabezados_objetivo.issubset(set(encabezados))

def scrapPDFBibliography(ruta, encabezados_objetivo):
    """
    Extrae una tabla específica desde un PDF que contiene los encabezados objetivo.

    Parámetros:
        ruta (str): Ruta del archivo PDF.
        encabezados_objetivo (set[str]): Encabezados que identifican la tabla buscada.

    Retorna:
        pandas.DataFrame: Tabla encontrada, limpiada y combinada si se extiende en varias páginas.
    """
    tablas_por_pagina = []

    # Leer todas las tablas del PDF página por página
    with pdfplumber.open(ruta) as pdf:
        for pagina in pdf.pages:
            tablas_pagina = []
            for tabla in pagina.extract_tables():
                if tabla and len(tabla) > 1:
                    tablas_pagina.append(tabla)
            tablas_por_pagina.append(tablas_pagina)

    # Buscar la primera tabla que contenga los encabezados objetivo
    tabla_encontrada = None
    indice_pagina = -1
    indice_tabla = -1

    for i, tablas_pag in enumerate(tablas_por_pagina):
        for j, tabla in enumerate(tablas_pag):
            if hasTargetHeaders(tabla, encabezados_objetivo):
                tabla_encontrada = tabla
                indice_pagina = i
                indice_tabla = j
                break
        if tabla_encontrada:
            break
    
    # No se encontró tabla válida
    if not tabla_encontrada:
        return pd.DataFrame()  

    # Combinar con tablas de páginas siguientes si es necesario
    filas_combinadas = list(tabla_encontrada)
    pagina_actual = indice_pagina

    while True:
        tablas_pag_actual = tablas_por_pagina[pagina_actual]

        # Verificar si la tabla actual es la última de la página
        if indice_tabla != len(tablas_pag_actual) - 1:
            break

        # Verificar si hay una página siguiente
        if pagina_actual + 1 >= len(tablas_por_pagina):
            break

        # Verificar si la siguiente página contiene tablas
        tablas_pag_siguiente = tablas_por_pagina[pagina_actual + 1]
        if not tablas_pag_siguiente:
            break

        # Tomar la primera tabla de la siguiente página
        primera_tabla_siguiente = tablas_pag_siguiente[0]

        # Si la siguiente tabla vuelve a tener encabezados, no combinar
        if hasTargetHeaders(primera_tabla_siguiente, encabezados_objetivo):
            break

        # Combinar las filas (sin encabezado repetido)
        filas_combinadas.extend(primera_tabla_siguiente)
        pagina_actual += 1
        indice_tabla = 0

    # Crear DataFrame
    df = pd.DataFrame(filas_combinadas[1:], columns=filas_combinadas[0])

    # Limpiar filas no deseadas
    df = df[~df["Nombre"].str.contains(r"^https?://", na=False)]
    df = df[df["Tipo"] == "Bibliografía"]

    # Eliminar saltos de línea en columnas
    df["Nombre"] = df["Nombre"].str.replace("\n", " ", regex=False)
    df["Observaciones"] = df["Observaciones"].str.replace("\n", " ", regex=False)

    return df

def scrapGoogleScholar(name):
    """
    Busca un autor o título en Google Scholar y devuelve información básica del primer resultado.

    Parámetros:
        name (str): Nombre del autor o texto a buscar.

    Retorna:
        dict: Contiene título, autores y enlace del primer resultado encontrado.
    """
    url = f"https://scholar.google.com/scholar?q={quote(name)}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    respuesta = requests.get(url, headers=headers)

    if respuesta.status_code != 200:
        raise Exception(f"Error al acceder a Google Scholar: {respuesta.status_code}")

    soup = BeautifulSoup(respuesta.text, "html.parser")
    resultado = {}
    item = soup.select_one(".gs_r.gs_or.gs_scl")

    if item:
        titulo_elem = item.select_one(".gs_rt")
        autor_elem = item.select_one(".gs_a")
        link_elem = titulo_elem.find("a") if titulo_elem else None

        titulo = titulo_elem.get_text(strip=True) if titulo_elem else ""
        autores = autor_elem.get_text(strip=True) if autor_elem else ""
        enlace = link_elem["href"] if link_elem and link_elem.has_attr("href") else ""

        # Limpiar etiquetas comunes
        for tag in ["[PDF]", "[LIBRO]", "[B]", "[CITAS]", "[C]", "[HTML]"]:
            titulo = titulo.replace(tag, "")

        resultado["Título"] = titulo.strip()
        resultado["Autores"] = autores.split("-")[0].strip()
        resultado["Enlace"] = enlace

    return resultado


In [ ]:
directory = "Guias Docentes"

dfs_scrap = []

for file in os.listdir(directory):
    if file.endswith(".pdf"):  
        ruta = os.path.join(directory, file)
        df = scrapPDFBibliography(ruta, {"nombre", "tipo", "observaciones"})  
        if not df.empty:  
            df["Guia Docente"] = file  
            dfs_scrap.append(df)

df_scrap = pd.concat(dfs_scrap, ignore_index=True)

dfs_google = []

for nombre in df_scrap['Nombre']:
    try:
        dict_bibliografia = scrapGoogleScholar(nombre)
        df_temp = pd.DataFrame([dict_bibliografia])
        dfs_google.append(df_temp)
    except Exception as e:
        print(f"Error con {nombre}: {e}")

    time.sleep(random.uniform(5, 15))

if dfs_google != []:
    df_google = pd.concat(dfs_google, ignore_index=True)
    df_google.head()


# Conectar a PostGreSQL

CREATE TABLE Asignaturas (
    id SERIAL PRIMARY KEY,
    nombre VARCHAR(150) NOT NULL,
    coordinador VARCHAR(100) NOT NULL,
    numero_creditos INT NOT NULL,
    agno_academico VARCHAR(100) NOT NULL,
    direccion_url VARCHAR(255) NOT NULL,
    semestre VARCHAR(100) NOT NULL,
    idioma VARCHAR(100) NOT NULL,
    id_guia_docente INT NOT NULL,
    titulacion INT NOT NULL,
    FOREIGN KEY (titulacion) REFERENCES Titulaciones(id)
);


CREATE TABLE Bibliografias (
    id SERIAL PRIMARY KEY,
    asignatura_id INT, 
    nombre VARCHAR(100),
    autores VARCHAR(100),
    url VARCHAR(200),
    FOREIGN KEY (asignatura_id) REFERENCES Asignaturas(id)  
);


In [ ]:
from sqlalchemy import create_engine

usuario = "userPSQL"
contraseña = "passPSQL"
host = "localhost"  
puerto = "5432"
base_datos = "postgres"

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
)

df_google.to_sql('Bibliografias', engine, if_exists="replace", index=False)


5